In [47]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [39]:
sim.simxFinish(-1)  # Close opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    print('Simulation time in milliseconds: ', sim.simxGetLastCmdTime(clientID))
    
    # Get Object position
    name = 'Omnirob'
    err_code, cuboid = sim.simxGetObjectHandle(clientID, name, sim.simx_opmode_blocking)
    res, position = sim.simxGetObjectPosition(clientID, cuboid, -1, sim.simx_opmode_blocking)        
    print('Omnirob is at [x,y,z]=', position)
    
    # Now close the connection to CoppeliaSim:
    sim.simxGetPingTime(clientID)
    sim.simxFinish(clientID)
    print('Disconnected')
else:
    print('Failed connecting to remote API server')

Connected
Simulation time in milliseconds:  24200
Omnirob is at [x,y,z]= [-5.0001044273376465, -7.001952171325684, 0.35244935750961304]
Disconnected


In [48]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)

In [22]:
# Joint name in coppelia
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
r = robot('Omnirob', motor_names)  # Create an instance of our robot

# Initial locations
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

# desired_position = r.get_object_position('Sphere1')
# print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

# relative_position = r.get_object_relative_position('Sphere1')
# print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

desired_position = [10.00, -7.00, 0]

# Virtual disk
disk = robot('Disc', client_id=r.client_id)


# Time duration
tf = 10
# Time interval
time_steps = linspace(0, tf, 500)

a0 = robot_position
a1 = [1, -1, 0]
a2 = (np.dot(3,desired_position)-3*robot_position-np.dot(a1,2*tf))/tf**2
a3 = (2*robot_position+np.dot(a1,tf)-np.dot(2,desired_position))/tf**3

for t in time_steps:
    point_traj = a0 + np.dot(a1,t) + np.dot(a2,t*t) + np.dot(a3,t*t*t)
    vel_traj = a1 + 2*a2*t + 3*a3*t*t
    print(point_traj)
    
    disk.set_position(point_traj)
    
    
    # Sensing
    robot_position = r.get_position()
    
         
    # Trajectory tracker
    u = 10 * (point_traj - robot_position) + vel_traj
    
    vx, vy, vz = u
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    
    
    
    time.sleep(tf/500)

r.send_motor_velocities([0, 0, 0, 0])

r.close_connection() 

Robot connected
Robot position: (-5.00, -7.06) 
Robot connected
[-5.00360537 -7.06473017  0.35244885]
[-4.983465   -7.08468923  0.35244461]
[-4.96312472 -7.10448658  0.35243191]
[-4.94258547 -7.1241227   0.35241079]
[-4.92184824 -7.14359809  0.35238127]
[-4.90091398 -7.16291324  0.3523434 ]
[-4.87978366 -7.18206863  0.35229721]
[-4.85845825 -7.20106475  0.35224273]
[-4.83693871 -7.2199021   0.35217999]
[-4.81522601 -7.23858115  0.35210903]
[-4.79332112 -7.25710241  0.35202989]
[-4.771225   -7.27546636  0.35194259]
[-4.74893862 -7.29367348  0.35184718]
[-4.72646294 -7.31172428  0.35174368]
[-4.70379894 -7.32961923  0.35163213]
[-4.68094757 -7.34735882  0.35151257]
[-4.6579098  -7.36494356  0.35138502]
[-4.6346866  -7.38237391  0.35124953]
[-4.61127894 -7.39965038  0.35110612]
[-4.58768778 -7.41677346  0.35095483]
[-4.56391408 -7.43374362  0.35079569]
[-4.53995882 -7.45056137  0.35062875]
[-4.51582296 -7.46722719  0.35045402]
[-4.49150746 -7.48374157  0.35027155]
[-4.4670133  -7.50010499

[ 2.51131085 -8.42011269  0.20837716]
[ 2.55217186 -8.41633303  0.20733301]
[ 2.59302121 -8.41249879  0.20628786]
[ 2.63385794 -8.40861045  0.20524174]
[ 2.67468108 -8.4046685   0.20419469]
[ 2.71548965 -8.40067342  0.20314673]
[ 2.75628271 -8.39662571  0.2020979 ]
[ 2.79705927 -8.39252586  0.20104824]
[ 2.83781838 -8.38837435  0.19999778]
[ 2.87855906 -8.38417167  0.19894655]
[ 2.91928036 -8.37991832  0.19789459]
[ 2.9599813  -8.37561478  0.19684194]
[ 3.00066092 -8.37126154  0.19578862]
[ 3.04131825 -8.36685909  0.19473467]
[ 3.08195233 -8.36240792  0.19368012]
[ 3.12256219 -8.35790852  0.19262501]
[ 3.16314687 -8.35336137  0.19156938]
[ 3.20370539 -8.34876698  0.19051325]
[ 3.2442368  -8.34412582  0.18945666]
[ 3.28474012 -8.33943838  0.18839965]
[ 3.32521439 -8.33470516  0.18734225]
[ 3.36565865 -8.32992664  0.18628448]
[ 3.40607192 -8.32510332  0.1852264 ]
[ 3.44645325 -8.32023568  0.18416802]
[ 3.48680166 -8.3153242   0.18310939]
[ 3.52711619 -8.31036939  0.18205054]
[ 3.56739587

[ 9.54362408 -7.12577011  0.01361641]
[ 9.55842167 -7.12181709  0.01317784]
[ 9.57299408 -7.11791758  0.01274578]
[ 9.58734036 -7.11407208  0.01232027]
[ 9.60145952 -7.11028106  0.01190135]
[ 9.61535061 -7.10654502  0.01148905]
[ 9.62901267 -7.10286444  0.01108341]
[ 9.64244471 -7.09923983  0.01068445]
[ 9.65564579 -7.09567166  0.01029222]
[ 9.66861493 -7.09216042  0.00990674]
[ 9.68135116e+00 -7.08870660e+00  9.52805678e-03]
[ 9.69385353e+00 -7.08531070e+00  9.15619582e-03]
[ 9.70612106e+00 -7.08197320e+00  8.79119370e-03]
[ 9.71815279e+00 -7.07869460e+00  8.43308445e-03]
[ 9.72994775e+00 -7.07547537e+00  8.08190213e-03]
[ 9.74150497e+00 -7.07231601e+00  7.73768077e-03]
[ 9.75282350e+00 -7.06921701e+00  7.40045440e-03]
[ 9.76390235e+00 -7.06617886e+00  7.07025707e-03]
[ 9.77474058e+00 -7.06320204e+00  6.74712281e-03]
[ 9.78533720e+00 -7.06028705e+00  6.43108567e-03]
[ 9.79569127e+00 -7.05743437e+00  6.12217968e-03]
[ 9.80580180e+00 -7.05464450e+00  5.82043888e-03]
[ 9.81566783e+00 -7.

In [49]:
def move(desired_position, init_velocity):
    # Joint name in coppelia
    motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
    r = robot('Omnirob', motor_names)  # Create an instance of our robot

    # Initial locations
    robot_position = r.get_position()
#     print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

    # desired_position = r.get_object_position('Sphere1')
    # print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

    # relative_position = r.get_object_relative_position('Sphere1')
    # print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

#     desired_position = [10.00, -7.00, 0]

    # Virtual disk
    disk = robot('Disc', client_id=r.client_id)


    # Time duration
    tf = 10
    # Time interval
    time_steps = linspace(0, tf, 500)

    a0 = robot_position
#     a1 = [1, -1, 0]
    a1 = init_velocity
    a2 = (np.dot(3,desired_position)-3*robot_position-np.dot(a1,2*tf))/tf**2
    a3 = (2*robot_position+np.dot(a1,tf)-np.dot(2,desired_position))/tf**3

    for t in time_steps:
        point_traj = a0 + np.dot(a1,t) + np.dot(a2,t*t) + np.dot(a3,t*t*t)
        vel_traj = a1 + 2*a2*t + 3*a3*t*t
#         print(point_traj)

        disk.set_position(point_traj)


        # Sensing
        robot_position = r.get_position()


        # Trajectory tracker
        u = 10 * (point_traj - robot_position) + vel_traj

        vx, vy, vz = u
        r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])



        time.sleep(tf/500)

    r.send_motor_velocities([0, 0, 0, 0])

    r.close_connection() 

In [53]:
move((10, -7, 0),(1,-1,0))
move((10, -3, 0),(1,1,0))
move((-1, 0, 0),(-1,0,0))
move((3.5, 10, 0),(-0.2,1.5,0))
move((3.5, 0, 0),(0,0,0))
move((-1, 2, 0),(-1,-1,0))
move((2, 10, 0),(0,1,0))
move((9, 10, 0),(1,1,0))

Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
